# Повторение примеров

## Julia для науки о данных

### Считывание данных

In [1]:
using CSV, DataFrames, DelimitedFiles

# Считывание данных и их запись в структуру:
P = CSV.File("programminglanguages.csv") |> DataFrame
# Функция определения по названию языка программирования года его создания:
function language_created_year(P,language::String)
    loc = findfirst(P[:,2].==language)
    return P[loc,1]
end
# Пример вызова функции и определение даты создания языка Python:
language_created_year(P,"Python")
# Пример вызова функции и определение даты создания языка Julia:
language_created_year(P,"Julia")

In [ ]:
language_created_year(P,"julia")

In [ ]:
# Функция определения по названию языка программирования года его создания (без учёта регистра):
function language_created_year_v2(P,language::String)
loc = findfirst(lowercase.(P[:,2]).==lowercase.(language))
return P[loc,1]
end
# Пример вызова функции и определение даты создания языка julia:
language_created_year_v2(P,"julia")

In [ ]:
# Построчное считывание данных с указанием разделителя:
Tx = readdlm("programminglanguages.csv", ',')

### Запись данных в файл

In [ ]:
# Запись данных в CSV-файл:
CSV.write("programming_languages_data2.csv", P)

In [ ]:
# Пример записи данных в текстовый файл с разделителем ',':
writedlm("programming_languages_data.txt", Tx, ',')

In [ ]:
# Пример записи данных в текстовый файл с разделителем '-':
writedlm("programming_languages_data2.txt", Tx, '-')

In [ ]:
# Построчное считывание данных с указанием разделителя:
P_new_delim = readdlm("programming_languages_data2.txt", '-')

### Словари

In [ ]:
# Инициализация словаря:
dict = Dict{Integer,Vector{String}}()

In [ ]:
# Инициализация словаря:
dict2 = Dict()

In [ ]:
# Заполнение словаря данными:
for i = 1:size(P,1)
    year,lang = P[i,:]
    if year in keys(dict)
        dict[year] = push!(dict[year],lang)
    else
        dict[year] = [lang]
    end
end
dict

In [ ]:
# Пример определения в словаре языков программирования, созданных в 2003 году:
dict[2003]

### DataFrames

In [ ]:
# Задаём переменную со структурой DataFrame:
df = DataFrame(year = P[:,1], language = P[:,2])
# Вывод всех значения столбца year:
df[!,:year]

In [ ]:
# Получение статистических сведений о фрейме:
describe(df)

### RDatasets

In [ ]:
# Подгружаем пакет RDatasets:
using RDatasets
# Задаём структуру данных в виде набора данных:
iris = dataset("datasets", "iris")

In [ ]:
# Определения типа переменной:
typeof(iris)

In [ ]:
describe(iris)

### Работа с переменными отсутствующего типа (Missing Values)

In [ ]:
# Отсутствующий тип:
a = missing
typeof(a)

In [ ]:
# Пример операции с переменной отсутствующего типа:
a + 1

In [ ]:
# Определение перечня продуктов:
foods = ["apple", "cucumber", "tomato", "banana"]
# Определение калорий:
calories = [missing,47,22,105]

In [ ]:
# Определение типа переменной:
typeof(calories)

In [ ]:
# Подключаем пакет Statistics:
using Statistics
# Определение среднего значения:
mean(calories)

In [ ]:
# Определение среднего значения без значений с отсутствующим типом:
mean(skipmissing(calories))

In [ ]:
# Задание сведений о ценах:
prices = [0.85,1.6,0.8,0.6]
# Формирование данных о калориях:
dataframe_calories = DataFrame(item=foods,calories=calories)
# Формирование данных о ценах:
dataframe_prices = DataFrame(item=foods,price=prices)
# Объединение данных о калориях и ценах:
DF = join(dataframe_calories,dataframe_prices,on=:item)

### FileIO

In [ ]:
# Подключаем пакет FileIO:
using FileIO
using ImageIO
# Загрузка изображения:
X1 = load("julialogo.png")

In [ ]:
# Определение типа и размера данных:
@show typeof(X1);
@show size(X1);

## Обработка данных: стандартные алгоритмы машинного обучения в Julia

### Кластеризация данных. Метод k-средних

In [ ]:
using Plots
# Загрузка данных:
houses = CSV.File("houses.csv") |> DataFrame

In [ ]:
plot(size=(500,500),leg=false)
x = houses[!,:sq__ft]
y = houses[!,:price]
scatter(x,y,markersize=3)

In [ ]:
# Фильтрация данных по заданному условию:
filter_houses = houses[houses[!,:sq__ft].>0,:]
# Построение графика:
x = filter_houses[!,:sq__ft]
y = filter_houses[!,:price]
scatter(x,y)

In [ ]:
# Определение средней цены для определённого типа домов:
by(filter_houses,:type,filter_houses->mean(filter_houses[!,:price]))

In [ ]:
using Clustering
# Добавление данных :latitude и :longitude в новый фрейм:
X = filter_houses[[:latitude,:longitude]]
# Конвертация данных в матричный вид:
X = convert(Matrix{Float64}, X)

In [ ]:
# Транспонирование матрицы с данными:
X = X'
# Задание количества кластеров:
k = length(unique(filter_houses[!,:zip]))
# Определение k-среднего:
C = kmeans(X,k)

In [ ]:
# Формирование фрейма данных:
df = DataFrame(cluster = C.assignments,city = filter_houses[!,:city], latitude = filter_houses[!,:latitude],longitude = filter_houses[!,:longitude],zip = filter_houses[!,:zip])

In [ ]:
clusters_figure = plot(legend = false)
for i = 1:k
clustered_houses = df[df[!,:cluster].== i,:]
xvals = clustered_houses[!,:latitude]
yvals = clustered_houses[!,:longitude]
scatter!(clusters_figure,xvals,yvals,markersize=4)
end
xlabel!("Latitude")
ylabel!("Longitude")
title!("Houses color-coded by cluster")
display(clusters_figure)

In [ ]:
unique_zips = unique(filter_houses[!,:zip])
zips_figure = plot(legend = false)
for uzip in unique_zips
subs = filter_houses[filter_houses[!,:zip].==uzip,:]
x = subs[!,:latitude]
y = subs[!,:longitude]
scatter!(zips_figure,x,y)
end
xlabel!("Latitude")
ylabel!("Longitude")
title!("Houses color-coded by zip code")
display(zips_figure)

### Кластеризация данных. Метод k ближайших соседей

In [ ]:
using NearestNeighbors
knearest = 10
id = 70
point = X[:,id]
# Поиск ближайших соседей:
kdtree = KDTree(X)
idxs, dists = knn(kdtree, point, knearest, true)
# Все объекты недвижимости:
x = filter_houses[!,:latitude];
y = filter_houses[!,:longitude];
scatter(x,y)
# Соседи:
x = filter_houses[idxs,:latitude];
y = filter_houses[idxs,:longitude];
scatter!(x,y)

In [ ]:
# Фильтрация по районам соседних домов:
cities = filter_houses[idxs,:city]

### Обработка данных. Метод главных компонент

In [ ]:
# Фрейм с указанием площади и цены недвижимости:
F = filter_houses[[:sq__ft,:price]]
# Конвертация данных в массив:
F = convert(Array{Float64,2},F)'
# Подключение пакета MultivariateStats:
using MultivariateStats
# Приведение типов данных к распределению для PCA:
M = fit(PCA, F)
# Выделение значений главных компонент в отдельную переменную:
Xr = reconstruct(M, y)
# Построение графика с выделением главных компонент:
scatter(F[1,:],F[2,:])
scatter!(Xr[1,:],Xr[2,:])

### Обработка данных. Линейная регрессия

In [ ]:
xvals = repeat(1:0.5:10,inner=2)
yvals = 3 .+ xvals + 2*rand(length(xvals)) .- 1
scatter(xvals,yvals,color=:black,leg=false)

In [ ]:
function find_best_fit(xvals,yvals)
meanx = mean(xvals)
meany = mean(yvals)
stdx = std(xvals)
stdy = std(yvals)
r = cor(xvals,yvals)
a = r*stdy/stdx
b = meany - a*meanx
return a,b
end

In [ ]:
a,b = find_best_fit(xvals,yvals)
ynew = a * xvals .+ b
plot!(xvals,ynew)

In [ ]:
xvals = 1:100000;
xvals = repeat(xvals,inner=3);
yvals = 3 .+ xvals + 2*rand(length(xvals)) .- 1;
@show size(xvals)
@show size(yvals)
@time a,b = find_best_fit(xvals,yvals)

In [ ]:
using PyCall
using Conda
py"""
import numpy
def find_best_fit_python(xvals,yvals):
meanx = numpy.mean(xvals)
meany = numpy.mean(yvals)
stdx = numpy.std(xvals)
stdy = numpy.std(yvals)
r = numpy.corrcoef(xvals,yvals)[0][1]
a = r*stdy/stdx
b = meany - a*meanx
return a,b
"""
xpy = PyObject(xvals)
ypy = PyObject(yvals)
@time a,b = find_best_fit_python(xpy,ypy)

In [ ]:
using BenchmarkTools
@btime a,b = find_best_fit_python(xvals,yvals)
@btime a,b = find_best_fit(xvals,yvals)

# Самостоятельная работа

## Кластеризация

Используйте Clustering.jl для кластеризации на основе k-средних. Сделайте точечную диаграмму полученных кластеров.
Подсказка: вам нужно будет проиндексировать фрейм данных, преобразовать его в массив и транспонировать.

## Регрессия (метод наименьших квадратов в случае линейной регрессии)

Часть 1.

Часть 2.

## Модель ценообразования биномиальных опционов